# Naive Bayes With Natural Language Processing

In this demo, we will look at using the `tm` (text mining) package to perform Natural Language Processing (NLP) on a data set and generate a Naive Bayes model which performs sentiment analysis.

To do this, we will need four packages:
* The `naivebayes` package will help us generate our model.
* The `tidyverse` package gives us the `magrittr` pipe.  In a real scenario, we'd probably use other `tidyverse` functionality like `dplyr` functions or `ggplot2` graphics.
* The `tm` package allows us to perform text mining activities.  An alternative package would be `tidytext`.
* The `caret` package gives us a good confusion matrix.

In [ ]:
if(!require(naivebayes)) {
  install.packages("naivebayes")
  library(naivebayes)
}
if(!require(tidyverse)) {
  install.packages("tidyverse")
  library(tidyverse)
}
if(!require(tm)) {
  install.packages("tm")
  library(tm)
}
if(!require(caret)) {
  install.packages("caret")
  library(caret)
}

Our data set is a sample of 2000 movie reviews.  There are two variables in this data set:  a **class** feature which is "Pos" (for a positive review) and "Neg" (for a negative review).  Then, **text** is the review itself.

The data set is still a bit messy, so we will have to do some work on it to simplify processing.

In [ ]:
df <- readr::read_csv("../data/movie-pang02.csv")

In [ ]:
glimpse(df)

Just like in the first demo, the data set size is small enough that I don't mind shuffling data this way.  I do want to make sure that **class** is a factor, however.

In [ ]:
set.seed(1)
df <- df[sample(nrow(df)),]
df <- df[sample(nrow(df)),]
df$class <- as.factor(df$class)

Our first step after shuffling data is to build a corpus.  This is the full set of data.

**NOTE:** in a real data science project, I would *not* build the corpus here; I'd split into training and test data sets.  Then I'd build a corpus for the training set and apply those same rules to the test data set independently.  The problem is that you have the potential of some information from the test set leaking into the training set, which might cause overfitting against the test set.  This is troublesome because you'll think you have a good accuracy, but accuracy might drop considerably when moving to real-world data. 

The reason why we might think about doing this, though, is that it simplifies the document term matrix that we end up creating--it prevents the possibility of a term showing up in the test data set which did not appear in the original DTM we used for training.  This is a sloppy way of getting around a potential error during testing.

In [ ]:
corpus <- tm::Corpus(tm::VectorSource(df$text))
corpus

We are applying a number of rules to the data.  To simplify analysis, we convert capital letters to lower-case (to make sure that any case-sensitive string comparisons work).  Then we remove punctuation, numbers, stopwords, and extraneous whitespace.  This gives us a much simpler data set to work with.

In [ ]:
corpus.clean <- corpus %>%
  tm::tm_map(tm::content_transformer(tolower)) %>% 
  tm::tm_map(tm::removePunctuation) %>%
  tm::tm_map(tm::removeNumbers) %>%
  tm::tm_map(tm::removeWords, tm::stopwords(kind="en")) %>%
  tm::tm_map(tm::stripWhitespace)

This next bit is the real trick in NLP:  converting words to features.  We're going to use the bag of words technique, converting our set of unique words into a set of features, and building a matrix of reviews versus words, with the elements being count of occurrences.

Below is a sample of 6 terms as they appear in 11 documents (that is, our reviews).

In [ ]:
dtm <- tm::DocumentTermMatrix(corpus.clean)
inspect(dtm[40:50,10:15])

From here, we split out training and test data.  Please see above about how this can lead to some overfitting of the test data.

Our split is 75% training and 25% test data.  You can experiment with other percentages as well.

In [ ]:
df.train <- df[1:1500,]
df.test <- df[1501:2000,]

dtm.train <- dtm[1:1500,]
dtm.test <- dtm[1501:2000,]

corpus.clean.train <- corpus.clean[1:1500]
corpus.clean.test <- corpus.clean[1501:2000]

Checking our input data set, there are 38,957 unique terms.

In [ ]:
dim(dtm.train)

Many of these terms appear in only one document.  When using a technique like TF-IDF, that can be a good thing, but if you have a huge number of terms, it can quickly blow out your available memory.  We'll limit our input set to a set of frequently-used terms.  In this case, we define "frequently used" as existing in five separate documents.

Once we do that, we're down to 12,144 unique terms.

In [ ]:
fiveFreq <- tm::findFreqTerms(dtm.train, 5)
length(fiveFreq)

Now that our dictionary has changed (from the full set of terms to our frequently-used terms), we'll want to build a new pair of document term matrices, feeding in the frequently-used terms as our dictionary.

In [ ]:
dtm.train.nb <- tm::DocumentTermMatrix(corpus.clean.train, control=list(dictionary = fiveFreq))
dtm.test.nb <- tm::DocumentTermMatrix(corpus.clean.test, control=list(dictionary = fiveFreq))

The number of occurrences of a term can be interesting, but in this analysis, we'll simplify the calculations a bit by looking for the *appearance* of a term rather than *number of appearances*.  If the term "available" shows up 30 times in one document, that counts as 1 occurrence rather than 30.

In [ ]:
convert_count <- function(x) {
  y <- ifelse(x > 0, 1,0)
  y <- factor(y, levels=c(0,1), labels=c("No", "Yes"))
  y
}

trainNB <- apply(dtm.train.nb, 2, convert_count)
testNB <- apply(dtm.test.nb, 2, convert_count)

Now that our inputs are cleaned up and ready to go, let's build a classifier function and then predict using that classifier against the test data.

In [ ]:
classifier <- naivebayes::naive_bayes(trainNB, df.train$class, laplace = 1)
pred <- predict(classifier, newdata=testNB)

Once we've done that, we have a new confusion matrix.  We can see that our classifier has an accuracy of 81%.

This example lets us look at four important features:  sensitivity, specificity, positive predictive value, and negative predictive value.  This is a simple two-class, binary classifier so these definitions are pretty simple.  The tricky part is that the confusion matrix in `caret` orders alphabetically, whereas ideally you want the "positive" result first and the "negative" result last. 

**Sensitivity** is where we capture when an event is positive, whether our predictor considers it positive, and is defined as (Rpos|Ppos) / (Rpos).  That is, 181/(181+54) or 0.7702.

**Specificity** is where we capture when an event is negative, whether our predictor considers it negative, and is defined as (Rneg|Pneg) / (Rneg).  That is, 224/(224+41) or 0.8453.

**Positive predictive value** looks at all cases where the Prediction was positive (read the "Pos" row), and is defined as (Ppos|Rpos) / (Ppos).  That is, 181/(181+41) or 0.8153.

**Negative predictive value** looks at cases where the Prediction was negative (read the "Neg" row), and is defined as (Pneg|Rneg) / (Pneg).  That is, 224/(224+54) or 0.8058.

In [ ]:
conf.mat <- caret::confusionMatrix(pred, df.test$class, positive="Pos")
conf.mat

We can also dig into specific elements of the confusion matrix like below:

In [ ]:
conf.mat$byClass

In [ ]:
conf.mat$overall

In [ ]:
conf.mat$overall["Accuracy"]